# Quickstart

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import climate_learn as cl
from climate_learn.data.climate_dataset.args import ERA5Args
from climate_learn.data.task.args import ForecastingArgs
from climate_learn.data.dataset.args import MapDatasetArgs

## Load the data

In [ ]:
root = "/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg/"
variables = ["geopotential_500"]#, "temperature_850", "2m_temperature"]
# variables = ['2m_temperature']
in_vars = out_vars = [f"era5:{v}" for v in variables]
train_years = range(1979, 2016)
val_years = range(2016, 2017)
test_years = range(2017, 2019)

forecasting_args = ForecastingArgs(
    in_vars,
    out_vars,
    pred_range=72,
    subsample=6
)

train_dataset_args = MapDatasetArgs(
    ERA5Args(root, variables, train_years),
    forecasting_args
)

val_dataset_args = MapDatasetArgs(
    ERA5Args(root, variables, val_years),
    forecasting_args
)

test_dataset_args = MapDatasetArgs(
    ERA5Args(root, variables, test_years),
    forecasting_args
)

dm = cl.data.DataModule(
    train_dataset_args,
    val_dataset_args,
    test_dataset_args,
    batch_size=32,
    num_workers=8
)

## Load the models

In [62]:
# climatology is the average value over the training period
# climatology = cl.load_forecasting_module(data_module=dm, preset="climatology")

# persistence returns its input as its prediction
# persistence = cl.load_forecasting_module(data_module=dm, preset="persistence")

# Rasp-Theurey 2020 is the SoTA on WeatherBench
# resnet = cl.load_forecasting_module(data_module=dm, preset="rasp-theurey-2020")

vit_pretrained = cl.load_forecasting_module(data_module=dm, 
    preset="vit", 
    use_pretrained_backbone=True, 
    use_pretrained_embeddings=True, 
    freeze_backbone=True,
    freeze_embeddings=True
)

Loading preset: vit
Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_mse
Loading test loss: lat_rmse
Loading test loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
No validation transform
Loading test transform: denormalize
Loading test transform: denormalize


## Looking at Rasp and Theurey training curves for reference

In [ ]:
rasp_theurey = cl.load_forecasting_module(data_module=dm, preset="rasp-theurey-2020")

In [ ]:
# change this to whatever gpu device you want to use
gpu_num = 0

import wandb
from pytorch_lightning.loggers import WandbLogger
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%H-%M-%S_%d-%m-%Y")

wandb.init(project="Climate", name=f'Rasp Theurey {dt_string}')
logger = WandbLogger()

trainer = cl.Trainer(
    # stop when latitude-weighted RMSE, a validation metric, stops improving
    early_stopping="lat_rmse:aggregate [val]",
    # wait for 1 epoch of no improvement
    patience=1,
    # uncomment to use gpu acceleration
    accelerator="gpu",
    devices=[gpu_num],
    # max epochs
    max_epochs=50,
    # log to wandb
    logger=logger,
    # Print model summary
    enable_model_summary=False,
)

In [44]:
trainer.fit(rasp_theurey, dm)

/home/rohanshah/miniconda3/envs/climate/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: 
UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

## Train the models

Climatology and persistence don't require training.

In [63]:
# change this to whatever gpu device you want to use
gpu_num = 0

import wandb
from pytorch_lightning.loggers import WandbLogger
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%H-%M-%S_%d-%m-%Y")

# wandb.init(project="Climate", name=f'VIT Pretrained {dt_string}')
# logger = WandbLogger()

trainer = cl.Trainer(
    # stop when latitude-weighted RMSE, a validation metric, stops improving
    early_stopping="lat_rmse:aggregate [val]",
    # wait for 1 epoch of no improvement
    patience=1,
    # uncomment to use gpu acceleration
    accelerator="gpu",
    devices=[gpu_num],
    # max epochs
    max_epochs=50,
    # log to wandb
    # logger=logger,
    # Print model summary
    enable_model_summary=False,
)

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅█████████████
lat_acc:aggregate [train]_epoch,▁▇█
lat_acc:aggregate [train]_step,▁▆▇▇▇▇▇▇▇▇█▇████████████████████████████
lat_acc:aggregate [val],▁██
lat_acc:era5:geopotential_500 [train]_epoch,▁▇█
lat_acc:era5:geopotential_500 [train]_step,▁▆▇▇▇▇▇▇▇▇█▇████████████████████████████
lat_acc:era5:geopotential_500 [val],▁██
lat_mse:aggregate [train]_epoch,█▂▁
lat_mse:aggregate [train]_step,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lat_mse:aggregate [val],█▁▁
lat_mse:era5:geopotential_500 [train]_epoch,█▂▁


/home/rohanshah/miniconda3/envs/climate/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Global seed set to 0
/home/rohanshah/climate-learn/src/climate_learn/trainer.py:48: UserWarning: In interactive environment: cannot use DDP spawn strategy
  warn("In interactive environment: cannot use DDP spawn strategy")


In [64]:

trainer.fit(vit_pretrained, dm)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                                                                  ┃ Type               ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                                                                   │ ViTPretrained      │  306 M │
│ 1   │ net.in_conv                                                           │ Conv2d             │      6 │
│ 2   │ net.forward_encoder                                                   │ ViTModel           │  304 M │
│ 3   │ net.forward_encoder.embeddings                                        │ ViTEmbeddings      │  990 K │
│ 4   │ net.forward_encoder.embeddings.patch_embeddings                       │ ViTPatchEmbeddings │  787 K │
│ 5   │ net.forward_encoder.embeddings.patch_embeddings.projection            │ Conv2d             │  787 K │
│ 6   │ net.forward_encoder.embeddings.dropout                                │ Dropout            │      0 │
│ 7   │ net.forward_encoder.encoder                                           │ ViTEncoder         │  302 M │
│ 8   │ net.forward_encoder.encoder.layer                                     │ ModuleList         │  302 M │
│ 9   │ net.forward_encoder.encoder.layer.0                                   │ ViTLayer           │ 12.6 M │
│ 10  │ net.forward_encoder.encoder.layer.0.attention                         │ ViTAttention       │  4.2 M │
│ 11  │ net.forward_encoder.encoder.layer.0.attention.attention               │ ViTSelfAttention   │  3.1 M │
│ 12  │ net.forward_encoder.encoder.layer.0.attention.attention.query         │ Linear             │  1.0 M │
│ 13  │ net.forward_encoder.encoder.layer.0.attention.attention.key           │ Linear             │  1.0 M │
│ 14  │ net.forward_encoder.encoder.layer.0.attention.attention.value         │ Linear             │  1.0 M │
│ 15  │ net.forward_encoder.encoder.layer.0.attention.attention.dropout       │ Dropout            │      0 │
│ 16  │ net.forward_encoder.encoder.layer.0.attention.output                  │ ViTSelfOutput      │  1.0 M │
│ 17  │ net.forward_encoder.encoder.layer.0.attention.output.dense            │ Linear             │  1.0 M │
│ 18  │ net.forward_encoder.encoder.layer.0.attention.output.dropout          │ Dropout            │      0 │
│ 19  │ net.forward_encoder.encoder.layer.0.intermediate                      │ ViTIntermediate    │  4.2 M │
│ 20  │ net.forward_encoder.encoder.layer.0.intermediate.dense                │ Linear             │  4.2 M │
│ 21  │ net.forward_encoder.encoder.layer.0.intermediate.intermediate_act_fn  │ GELUActivation     │      0 │
│ 22  │ net.forward_encoder.encoder.layer.0.output                            │ ViTOutput          │  4.2 M │
│ 23  │ net.forward_encoder.encoder.layer.0.output.dense                      │ Linear             │  4.2 M │
│ 24  │ net.forward_encoder.encoder.layer.0.output.dropout                    │ Dropout            │      0 │
│ 25  │ net.forward_encoder.encoder.layer.0.layernorm_before                  │ LayerNorm          │  2.0 K │
│ 26  │ net.forward_encoder.encoder.layer.0.layernorm_after                   │ LayerNorm          │  2.0 K │
│ 27  │ net.forward_encoder.encoder.layer.1                                   │ ViTLayer           │ 12.6 M │
│ 28  │ net.forward_encoder.encoder.layer.1.attention                         │ ViTAttention       │  4.2 M │
│ 29  │ net.forward_encoder.encoder.layer.1.attention.attention               │ ViTSelfAttention   │  3.1 M │
│ 30  │ net.forward_encoder.encoder.layer.1.attention.attention.query         │ Linear             │  1.0 M │
│ 31  │ net.forward_encoder.encoder.layer.1.attention.attention.key           │ Linear             │  1.0 M │
│ 32  │ net.forward_encoder.encoder.layer.1.attention.attention.value         │ Linear             │  1.0 M │
│ 33  │ net.forward_encoder.encoder.laye

Trainable params: 2.4 M                                                                                            
Non-trainable params: 304 M                                                                                        
Total params: 306 M                                                                                                
Total estimated model params size (MB): 1.2 K

Output()

Embedding Parameters

AttributeError: 'ViTModel' object has no attribute 'patch_embeddings'

## Test the models

In [ ]:
trainer.test(climatology, dm)

In [ ]:
trainer.test(persistence, dm)

In [ ]:
trainer.test(vit_pretrained, dm)